In [ ]:
from google.colab import drive

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re

https://colab.research.google.com/github/fastaudio/fastaudio/blob/master/docs/ESC50:%20Environmental%20Sound%20Classification.ipynb#scrollTo=bp4PfvDBAQWu

In [ ]:
# !pip install fastaudio

Restart runtime!

In [ ]:
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *
from fastaudio.ci import skip_if_ci

Due to receiving an `No module named 'torchvision.models.utils'` ModuleNotFoundError from torchvision, I opened and modified xresnet.py
`/usr/local/lib/python3.10/dist-packages/fastai/vision/models/xresnet.py in <module>`:

Change line 13 from `from torchvision.models.utils import load_state_dict_from_url` to `from torch.hub import load_state_dict_from_url`

In [ ]:
#The first time this will download a dataset that is ~650mb
# path = untar_data(URLs.ESC50, dest="ESC50")

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
src = '/content/drive/MyDrive/talkapalooza/labelled_audio'

In [ ]:
audio_data = get_files(src, extensions=audio_extensions)

In [ ]:
Path(src).ls()

(#471) [Path('/content/drive/MyDrive/talkapalooza/labelled_audio/yellow'),Path('/content/drive/MyDrive/talkapalooza/labelled_audio/big'),Path('/content/drive/MyDrive/talkapalooza/labelled_audio/nose'),Path('/content/drive/MyDrive/talkapalooza/labelled_audio/where'),Path('/content/drive/MyDrive/talkapalooza/labelled_audio/hear'),Path('/content/drive/MyDrive/talkapalooza/labelled_audio/hair'),Path('/content/drive/MyDrive/talkapalooza/labelled_audio/vacuum'),Path('/content/drive/MyDrive/talkapalooza/labelled_audio/bedroom'),Path('/content/drive/MyDrive/talkapalooza/labelled_audio/picture'),Path('/content/drive/MyDrive/talkapalooza/labelled_audio/above')...]

Recreate a dataframe of similar format:

In [ ]:
path = untar_data(URLs.ESC50, dest="ESC50")
df = pd.read_csv(path/"meta"/"esc50.csv")
df.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [ ]:
audio_df = pd.DataFrame(columns=['filepath', 'fold', 'label', 'author'])

In [ ]:
for f in audio_data:
  f_label = f.name.split('--')[0]
  f_author = f.name.split('--')[1][:-4]
  fpath = f"/{f_label}/{f.name}"
  temp_df = pd.DataFrame([[fpath, np.nan, f_label, f_author]], columns=['filepath', 'fold', 'label', 'author'])
  audio_df = pd.concat([audio_df, temp_df])

In [ ]:
audio_df.head()

,filepath,fold,label,author
0,/yellow/yellow--Back-ache.ogg,NaN,yellow,Back-ache
0,/yellow/yellow--Nattes-à-chat.ogg,NaN,yellow,Nattes-à-chat
0,/yellow/yellow--Lirazelf.ogg,NaN,yellow,Lirazelf
0,/big/big--Opsylac.ogg,NaN,big,Opsylac
0,/big/big--Lirazelf.ogg,NaN,big,Lirazelf


## Create Train and Test sets:

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

In [ ]:
np_array = audio_df.to_numpy()

In [ ]:
def generate_model_df (df_array):
  dfs = list()
  model = 1

  for train, test in kfold.split(df_array):
    temp_df = audio_df.copy().reset_index().drop(labels='index', axis=1).reset_index()
    temp_df['fold'] = model
    temp_df['test'] = 0

    # test_filt = (temp_df['index'].isin(test))
    # temp_df.loc[test_filt]['test'] = 1

    for i in test:
      temp_df.loc[i, 'test'] = 1

    temp_df.drop(labels='index', axis=1, inplace=True)

    dfs.append(temp_df)
    model = model+1

  return dfs

In [ ]:
dfs = generate_model_df(np_array)

In [ ]:
dfs[0].head(10)

,filepath,fold,label,author,test
0,/yellow/yellow--Back-ache.ogg,1,yellow,Back-ache,0
1,/yellow/yellow--Nattes-à-chat.ogg,1,yellow,Nattes-à-chat,0
2,/yellow/yellow--Lirazelf.ogg,1,yellow,Lirazelf,0
3,/big/big--Opsylac.ogg,1,big,Opsylac,0
4,/big/big--Lirazelf.ogg,1,big,Lirazelf,0
5,/big/big--Back-ache.ogg,1,big,Back-ache,0
6,/big/big--Greenman.ogg,1,big,Greenman,1
7,/big/big--SpringProof.ogg,1,big,SpringProof,0
8,/big/big--Vivekteraiyavt.ogg,1,big,Vivekteraiyavt,0
9,/big/big--She-animates.ogg,1,big,She-animates,0


## Datablock and Dataloader preparation:

Do I need this function, seeing as the different folds are already split into separate dataframes?

In [ ]:
# Helper function to split the data
def CrossValidationSplitter(col='fold', fold=1):
    "Split `items` (supposed to be a dataframe) by fold in `col`"
    def _inner(o):
        assert isinstance(o, pd.DataFrame), "ColSplitter only works when your items are a pandas DataFrame"
        col_values = o.iloc[:,col] if isinstance(col, int) else o[col]
        valid_idx = (col_values == fold).values.astype('bool')
        return IndexSplitter(mask2idxs(valid_idx))(o)
    return _inner

In [ ]:
# cfg = AudioConfig.Voice()
cfg = AudioConfig.BasicMelSpectrogram(n_fft=512)
a2s = AudioToSpec.from_cfg(cfg)

In [ ]:
auds = DataBlock(blocks = (AudioBlock, CategoryBlock),
                 get_x = ColReader("filepath", pref=src),
                 # splitter = CrossValidationSplitter(fold=1),
                 batch_tfms = [a2s],
                 get_y = ColReader("label"))

In [ ]:
dbunch = auds.dataloaders(dfs[0], bs=64)

/usr/local/lib/python3.10/dist-packages/fastaudio/augment/preprocess.py:86: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:363.)
  sig_np = ai.numpy()
/usr/local/lib/python3.10/dist-packages/fastaudio/augment/preprocess.py:91: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:363.)
  ai.data = torch.from_numpy(resampled.astype(np.float32))
/usr/local/lib/python3.10/dist-packages/fastaudio/augment/signal.py:202: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:363.)
  downmixed = ai.data.contiguous()

In [ ]:
dbunch.show_batch(figsize=(10, 5))

AttributeError: ignored